# Scraping Wikipedia about "Stumbling Stones" in Hamburg (Germany, Europe) – Part 2

#### In the first draft this project will be about "Stumbling Stones" (Stolpersteine) in the city of Hamburg where I live. In part 1 I started in my own neighborhood in the borough Wilhelmsburg. Now I will apply my scraper to the whole city
#### Later on I want to expand my project to every city of Germany and every other place where you can find them

In [408]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

## 1. Get urls for every "Stadtteil" (city district)

In [409]:
url_hamburg = "https://de.wikipedia.org/wiki/Liste_der_Stolpersteine_in_Hamburg"
html = requests.get(url_hamburg).text

In [410]:
# This code will not help me here because the tables with text I want to change are not on the first website

#html = html.replace("<br /><b>", " ")
#html = html.replace("</b><br />", " ")
#html = html.replace("<br />", " ")

In [411]:
soup = BeautifulSoup(html)

## 2. Select tables for each "Bezirk" (borough) and its "Stadtteile" (city districts)

#### Hamburg has 7 boroughs and 104 city districts

#### Find table for each borough

In [412]:
altona = soup.select("table.wikitable")[0]
bergedorf = soup.select("table.wikitable")[1]
eimsbüttel = soup.select("table.wikitable")[2]
hamburg_mitte = soup.select("table.wikitable")[3]
hamburg_nord = soup.select("table.wikitable")[4]
harburg = soup.select("table.wikitable")[5]
wandsbek = soup.select("table.wikitable")[6]

In [413]:
tables = [altona, bergedorf, eimsbüttel, hamburg_mitte, hamburg_nord, harburg, wandsbek]

#### Find all city districts with "Stumbling Stones"

##### (Not in all of the city districts are stones - I could also get the names of the NaN borough from the table)

In [414]:
all_districts = []
for table in tables:
    for cell in table.select("a"):
         all_districts.append(cell.text.strip())
        
len(all_districts)

80

## Got complete list of urls but had problems with results later because columns are different. Go on with each borough (Bezirk)

## (1) Altona

In [415]:
## len should be 13

districts_altona = []

for cell in altona.select("a"):
    district = cell.text.strip()
    url = "https://de.wikipedia.org" + cell["href"]          
    districts_altona.append({
        'Bezirk' : 'Altona',
        'Stadtteil' : district,
        'URL' : url
    }) 
len(districts_altona) 

13

In [416]:
districts_altona

table_list = []
for d in districts_altona:
    url = d['URL'] 
    html = requests.get(url).text
    soup = BeautifulSoup(html)
    d['Table'] = soup.select("table.wikitable")[0]
    d_table = d['Table']
    table_list.append(d_table)

table_list[1]

<table class="wikitable sortable toptextcells" style="width:100%;">
<tbody><tr>
<th>Adresse
</th>
<th>Person(en)
</th>
<th class="unsortable">Inschrift
</th>
<th class="unsortable">Bilder
</th>
<th class="unsortable">Anmerkung
</th></tr>
<tr>
<td data-sort-value="Bodenstedtstraße10">Bodenstedtstraße 10<br/><span class="plainlinks-print" id="Alfred_Abraham_Najmann" title="Koordinate"><a class="external text" href="https://geohack.toolforge.org/geohack.php?pagename=Liste_der_Stolpersteine_in_Hamburg-Altona-Nord&amp;language=de&amp;params=53.55794_N_9.94418_E_region:DE-HH_type:landmark&amp;title=Alfred+Abraham+Najmann"><span typeof="mw:File"><span><img class="mw-file-element" data-file-height="256" data-file-width="256" decoding="async" height="15" src="//upload.wikimedia.org/wikipedia/commons/thumb/9/9a/Erioll_world.svg/15px-Erioll_world.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/9/9a/Erioll_world.svg/23px-Erioll_world.svg.png 1.5x, //upload.wikimedia.org/wikipedia/c

In [417]:
districts_altona

districtlist, addresslist, namelist, inscriptionlist, picturelist, notelist = [],[],[],[],[],[]

count = 0

for table in table_list: 
    count = count + 1

    for row in table.select('tr')[1:]:
        cells = row.select("td")  
        addresslist.append(cells[0].text.strip())
        namelist.append(cells[1].text.strip())
        inscriptionlist.append(cells[2].text.strip())
        picturelist.append(cells[3].text.strip())
        notelist.append(cells[4].text.strip())
        districtlist.append(count)   

In [418]:
df = pd.DataFrame({'city' : 'Hamburg', 'borough' : 'Altona', 'district' : districtlist, 'address': addresslist, 'name': namelist, 'inscription': inscriptionlist, 'picture' : picturelist, 'note' : notelist})
df

,city,borough,district,address,name,inscription,picture,note
0,Hamburg,Altona,1,Amundsenstraße 14A,Max Cohn,Hier wohnte Max Cohn Jg. 1872 deportiert 1942 ...,,Eintrag auf stolpersteine-hamburg.de
1,Hamburg,Altona,1,Amundsenstraße 24,Friedrich W. Hoffmann,Hier wohnte Friedrich W. Hoffmann Jg. 1889 im ...,,Eintrag auf stolpersteine-hamburg.de
2,Hamburg,Altona,1,Behnstraße 17,Alfred Rosentreter,Hier wohnte Alfred Rosentreter Jg. 1934 deport...,,Eintrag auf stolpersteine-hamburg.de
3,Hamburg,Altona,1,Behnstraße 17,Betty Rosentreter,Hier wohnte Betty Rosentreter Jg. 1926 deporti...,,Eintrag auf stolpersteine-hamburg.de
4,Hamburg,Altona,1,Behnstraße 17,Fanny Rosentreter geb. Weinberger,Hier wohnte Fanny Rosentreter geb. Weinberger ...,,Eintrag auf stolpersteine-hamburg.de
...,...,...,...,...,...,...,...,...
521,Hamburg,Altona,13,Susannenstraße 4,Albert Löb,Hier wohnte Albert Löb Jg. 1889 deportiert 194...,,Eintrag auf stolpersteine-hamburg.de
522,Hamburg,Altona,13,Susannenstraße 6,Siegfried Salomon,Hier wohnte Siegfried Salomon Jg. 1882 deporti...,,Eintrag auf stolpersteine-hamburg.de
523,Hamburg,Altona,13,Susannenstraße 6,Ella Salomon geb. Karfiol,Hier wohnte Ella Salomon geb. Karfiol Jg. 1883...,,Eintrag auf stolpersteine-hamburg.de
524,Hamburg,Altona,13,Susannenstraße 40,Louise Rundstein geb. Klein,Hier wohnte Louise Rundstein geb. Klein Jg. 18...,,Eintrag auf stolpersteine-hamburg.de


In [419]:
from urllib.parse import urlparse, urlunparse

Geohack = []

for table in table_list: 
    for row in table.select('tr')[1:]:
        cells = []
        geo = []
        for cell in row.select("a"):
            cells.append(cell["href"])  
        for cell in cells:
            parsed_url = urlparse(cell)[4]
            geo.append(parsed_url)  
        for g in geo:
            if "pagename" in g:
                Geohack.append(g)
                
Geohack[:2]

['pagename=Liste_der_Stolpersteine_in_Hamburg-Altona-Altstadt&language=de&params=53.546928_N_9.948774_E_region:DE-HH_type:landmark&title=Max+Cohn',
 'pagename=Liste_der_Stolpersteine_in_Hamburg-Altona-Altstadt&language=de&params=53.546698_N_9.947123_E_region:DE-HH_type:landmark&title=Friedrich+W.+Hoffmann']

In [420]:
df['Geohack'] = Geohack
params = df['Geohack'].str.extract("params=(.*)region", expand=False)
df['lat'] = params.str.extract("(.*)_N", expand=False)
df['lon'] = params.str.extract("N_(.*)_E", expand=False)

df['district'] = df['Geohack'].str.extract("pagename=Liste_der_Stolpersteine_in_Hamburg-(.*)&language", expand=False)

df

,city,borough,district,address,name,inscription,picture,note,Geohack,lat,lon
0,Hamburg,Altona,Altona-Altstadt,Amundsenstraße 14A,Max Cohn,Hier wohnte Max Cohn Jg. 1872 deportiert 1942 ...,,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-Al...,53.546928,9.948774
1,Hamburg,Altona,Altona-Altstadt,Amundsenstraße 24,Friedrich W. Hoffmann,Hier wohnte Friedrich W. Hoffmann Jg. 1889 im ...,,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-Al...,53.546698,9.947123
2,Hamburg,Altona,Altona-Altstadt,Behnstraße 17,Alfred Rosentreter,Hier wohnte Alfred Rosentreter Jg. 1934 deport...,,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-Al...,53.5474654,9.9420098
3,Hamburg,Altona,Altona-Altstadt,Behnstraße 17,Betty Rosentreter,Hier wohnte Betty Rosentreter Jg. 1926 deporti...,,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-Al...,53.5474654,9.9420098
4,Hamburg,Altona,Altona-Altstadt,Behnstraße 17,Fanny Rosentreter geb. Weinberger,Hier wohnte Fanny Rosentreter geb. Weinberger ...,,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-Al...,53.5474654,9.9420098
...,...,...,...,...,...,...,...,...,...,...,...
521,Hamburg,Altona,Sternschanze,Susannenstraße 4,Albert Löb,Hier wohnte Albert Löb Jg. 1889 deportiert 194...,,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-St...,53.5619388,9.9626306
522,Hamburg,Altona,Sternschanze,Susannenstraße 6,Siegfried Salomon,Hier wohnte Siegfried Salomon Jg. 1882 deporti...,,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-St...,53.5620219,9.9628250
523,Hamburg,Altona,Sternschanze,Susannenstraße 6,Ella Salomon geb. Karfiol,Hier wohnte Ella Salomon geb. Karfiol Jg. 1883...,,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-St...,53.5620020,9.9627773
524,Hamburg,Altona,Sternschanze,Susannenstraße 40,Louise Rundstein geb. Klein,Hier wohnte Louise Rundstein geb. Klein Jg. 18...,,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-St...,53.562048,9.962618


In [421]:
df.to_csv('stolpersteine_altona.csv', index = False)

## (2) Bergedorf

In [422]:
### !!! Attention !!! The borough Bergedorf has two districts (Bergedorf and Lohbrügge), but they both have the same table. So we only need it once

In [423]:
## len should be 2, but the tables are the same (both boroughs are in one table, have the same link)

districts_bergedorf = []

for cell in bergedorf.select("a"):
    district = cell.text.strip()
    url = "https://de.wikipedia.org" + cell["href"]          
    districts_bergedorf.append({
        'Bezirk' : 'Bergedorf',
        'Stadtteil' : district,
        'URL' : url
    })   

print(districts_bergedorf)

[{'Bezirk': 'Bergedorf', 'Stadtteil': 'Bergedorf', 'URL': 'https://de.wikipedia.org/wiki/Liste_der_Stolpersteine_in_Hamburg-Bergedorf'}, {'Bezirk': 'Bergedorf', 'Stadtteil': 'Lohbrügge', 'URL': 'https://de.wikipedia.org/wiki/Liste_der_Stolpersteine_in_Hamburg-Bergedorf'}]


In [424]:
districts_bergedorf

table_list = []
for d in districts_bergedorf:
    url = d['URL'] 
    html = requests.get(url).text
    soup = BeautifulSoup(html)
    d['Table'] = soup.select("table.wikitable")[0]
    d_table = d['Table']
    table_list.append(d_table)

#table_list[1]
len(table_list)

2

In [425]:
districts_bergedorf

addresslist, namelist, inscriptionlist, picturelist, notelist = [],[],[],[],[]


for row in table_list[0].select('tr')[1:]:
    cells = row.select("td")  
    addresslist.append(cells[0].text.strip())
    namelist.append(cells[1].text.strip())
    inscriptionlist.append(cells[2].text.strip())
    picturelist.append(cells[3].text.strip())
    notelist.append(cells[4].text.strip())  

In [426]:
df = pd.DataFrame({'city' : 'Hamburg', 'borough' : 'Bergedorf', 'district' : 'Bergedorf und Lohbrügge', 'address': addresslist, 'name': namelist, 'inscription': inscriptionlist, 'picture' : picturelist, 'note' : notelist})
df.head(20)

,city,borough,district,address,name,inscription,picture,note
0,Hamburg,Bergedorf,Bergedorf und Lohbrügge,Alte Holstenstraße 61,Ernst Tichauer,Ernst Siegfried Tichauer (* 8. Oktober 1888 in...,2003,Stolpersteine für Ernst und Ellie Tichauer Woh...
1,Hamburg,Bergedorf,Bergedorf und Lohbrügge,Alte Holstenstraße 61,Ellie Tichauer,"Ellie Tichauer (* 6. November 1887 in Berlin, ...",2003,Stolperstein für Ellie Tichauer
2,Hamburg,Bergedorf,Bergedorf und Lohbrügge,August-Bebel-Straße 1,Mary Dobrzinsky,Mary Dobrzinski (* 1. Oktober 1880 in Bergedor...,2003,Stolperstein für Mary Dobrzinsky\nDie August-B...
3,Hamburg,Bergedorf,Bergedorf und Lohbrügge,August-Bebel-Straße 47,Vera Krebs,,2024,Stolperstein für Vera Krebs
4,Hamburg,Bergedorf,Bergedorf und Lohbrügge,August-Bebel-Straße 103,Claus Beeck,Claus Beeck (* 28. Januar 1940 in Bergedorf; †...,2013,"Stolperstein für Claus Beeck\nDie Gebäude, in ..."
5,Hamburg,Bergedorf,Bergedorf und Lohbrügge,August-Bebel-Straße 105,Margot Fischbeck,Margot Fischbeck (* 17. März 1935 in Bergedorf...,2013,Stolperstein für Margot Fischbeck
6,Hamburg,Bergedorf,Bergedorf und Lohbrügge,Chrysanderstraße 33,Marie Burke geb. Sponagel,Marie Karoline Dora Burke (* 17. April 1898 in...,2018,Stolperstein für Marie Burke
7,Hamburg,Bergedorf,Bergedorf und Lohbrügge,Ernst-Mantius-Straße 5,Hugo Rosendorff,Die fünf verlegten Stolpersteine erinnern an d...,2003,Stolpersteine für die ermordete Familie Rosend...
8,Hamburg,Bergedorf,Bergedorf und Lohbrügge,Ernst-Mantius-Straße 5,Hertha Rosendorff,"Hertha Rosendorff, (* 20. März 1882 in Hamburg...",2003,Stolperstein für Hertha Rosendorf
9,Hamburg,Bergedorf,Bergedorf und Lohbrügge,Ernst-Mantius-Straße 5,Inge Meier,Inge Meier (* 27. September 1917 in Hamburg; †...,2003,Stolperstein für Inge Meier


In [427]:
from urllib.parse import urlparse, urlunparse

Geohack = []


for row in table_list[0].select('tr')[1:]:
    cells = []
    geo = []
    for cell in row.select("a"):
        cells.append(cell["href"])  
    for cell in cells:
        parsed_url = urlparse(cell)[4]
        geo.append(parsed_url)  
    for g in geo:
        if "pagename" in g:
            Geohack.append(g)
                
Geohack[:2]

['pagename=Liste_der_Stolpersteine_in_Hamburg-Bergedorf&language=de&params=53.4900926_N_10.2090303_E_region:DE-HH_type:landmark&title=Dr.+Ernst+Tichauer%2C+Alte+Holstenstra%C3%9Fe+61',
 'pagename=Liste_der_Stolpersteine_in_Hamburg-Bergedorf&language=de&params=53.4900724_N_10.2090492_E_region:DE-HH_type:landmark&title=Ellie+Tichauer+geb.+Rosenthal%2C+Alte+Holstenstra%C3%9Fe+61']

In [428]:
df['Geohack'] = Geohack
params = df['Geohack'].str.extract("params=(.*)region", expand=False)
df['lat'] = params.str.extract("(.*)_N", expand=False)
df['lon'] = params.str.extract("N_(.*)_E", expand=False)

df['district'] = df['Geohack'].str.extract("pagename=Liste_der_Stolpersteine_in_Hamburg-(.*)&language", expand=False)

df

,city,borough,district,address,name,inscription,picture,note,Geohack,lat,lon
0,Hamburg,Bergedorf,Bergedorf,Alte Holstenstraße 61,Ernst Tichauer,Ernst Siegfried Tichauer (* 8. Oktober 1888 in...,2003,Stolpersteine für Ernst und Ellie Tichauer Woh...,pagename=Liste_der_Stolpersteine_in_Hamburg-Be...,53.4900926,10.2090303
1,Hamburg,Bergedorf,Bergedorf,Alte Holstenstraße 61,Ellie Tichauer,"Ellie Tichauer (* 6. November 1887 in Berlin, ...",2003,Stolperstein für Ellie Tichauer,pagename=Liste_der_Stolpersteine_in_Hamburg-Be...,53.4900724,10.2090492
2,Hamburg,Bergedorf,Bergedorf,August-Bebel-Straße 1,Mary Dobrzinsky,Mary Dobrzinski (* 1. Oktober 1880 in Bergedor...,2003,Stolperstein für Mary Dobrzinsky\nDie August-B...,pagename=Liste_der_Stolpersteine_in_Hamburg-Be...,53.4852604,10.2184875
3,Hamburg,Bergedorf,Bergedorf,August-Bebel-Straße 47,Vera Krebs,,2024,Stolperstein für Vera Krebs,pagename=Liste_der_Stolpersteine_in_Hamburg-Be...,53.485086,10.223984
4,Hamburg,Bergedorf,Bergedorf,August-Bebel-Straße 103,Claus Beeck,Claus Beeck (* 28. Januar 1940 in Bergedorf; †...,2013,"Stolperstein für Claus Beeck\nDie Gebäude, in ...",pagename=Liste_der_Stolpersteine_in_Hamburg-Be...,53.4842576,10.2290653
5,Hamburg,Bergedorf,Bergedorf,August-Bebel-Straße 105,Margot Fischbeck,Margot Fischbeck (* 17. März 1935 in Bergedorf...,2013,Stolperstein für Margot Fischbeck,pagename=Liste_der_Stolpersteine_in_Hamburg-Be...,53.4842490,10.2290779
6,Hamburg,Bergedorf,Bergedorf,Chrysanderstraße 33,Marie Burke geb. Sponagel,Marie Karoline Dora Burke (* 17. April 1898 in...,2018,Stolperstein für Marie Burke,pagename=Liste_der_Stolpersteine_in_Hamburg-Be...,53.488206,10.215314
7,Hamburg,Bergedorf,Bergedorf,Ernst-Mantius-Straße 5,Hugo Rosendorff,Die fünf verlegten Stolpersteine erinnern an d...,2003,Stolpersteine für die ermordete Familie Rosend...,pagename=Liste_der_Stolpersteine_in_Hamburg-Be...,53.4902116,10.2098612
8,Hamburg,Bergedorf,Bergedorf,Ernst-Mantius-Straße 5,Hertha Rosendorff,"Hertha Rosendorff, (* 20. März 1882 in Hamburg...",2003,Stolperstein für Hertha Rosendorf,pagename=Liste_der_Stolpersteine_in_Hamburg-Be...,53.4902367,10.2098989
9,Hamburg,Bergedorf,Bergedorf,Ernst-Mantius-Straße 5,Inge Meier,Inge Meier (* 27. September 1917 in Hamburg; †...,2003,Stolperstein für Inge Meier,pagename=Liste_der_Stolpersteine_in_Hamburg-Be...,53.4902548,10.2099544


In [429]:
df.to_csv('stolpersteine_bergedorf.csv', index = False)

## (3) Eimsbüttel

In [430]:
districts_eimsbüttel = []

for cell in eimsbüttel.select("a"):
    district = cell.text.strip()
    url = "https://de.wikipedia.org" + cell["href"]          
    districts_eimsbüttel.append({
        'Bezirk' : 'Eimsbüttel',
        'Stadtteil' : district,
        'URL' : url,
    })    
districts_eimsbüttel

[{'Bezirk': 'Eimsbüttel',
  'Stadtteil': 'Eidelstedt',
  'URL': 'https://de.wikipedia.org/wiki/Liste_der_Stolpersteine_in_Hamburg-Eidelstedt'},
 {'Bezirk': 'Eimsbüttel',
  'Stadtteil': 'Eimsbüttel',
  'URL': 'https://de.wikipedia.org/wiki/Liste_der_Stolpersteine_in_Hamburg-Eimsb%C3%BCttel'},
 {'Bezirk': 'Eimsbüttel',
  'Stadtteil': 'Harvestehude (Straßen A–H)',
  'URL': 'https://de.wikipedia.org/wiki/Liste_der_Stolpersteine_in_Hamburg-Harvestehude_(Stra%C3%9Fen_A%E2%80%93H)'},
 {'Bezirk': 'Eimsbüttel',
  'Stadtteil': 'Harvestehude (Straßen I–Z)',
  'URL': 'https://de.wikipedia.org/wiki/Liste_der_Stolpersteine_in_Hamburg-Harvestehude_(Stra%C3%9Fen_I%E2%80%93Z)'},
 {'Bezirk': 'Eimsbüttel',
  'Stadtteil': 'Hoheluft-West',
  'URL': 'https://de.wikipedia.org/wiki/Liste_der_Stolpersteine_in_Hamburg-Hoheluft-West'},
 {'Bezirk': 'Eimsbüttel',
  'Stadtteil': 'Lokstedt',
  'URL': 'https://de.wikipedia.org/wiki/Liste_der_Stolpersteine_in_Hamburg-Lokstedt'},
 {'Bezirk': 'Eimsbüttel',
  'Stadtteil'

In [431]:
districts_eimsbüttel

table_list = []
for d in districts_eimsbüttel:
    url = d['URL'] 
    html = requests.get(url).text
    soup = BeautifulSoup(html)
    d['Table'] = soup.select("table.wikitable")[0]
    d_table = d['Table']
    table_list.append(d_table)

#table_list[1]
len(table_list)

11

In [432]:
for row in table_list[0].select('tr')[1:]:
    cells = row.select("td")   
    print(cells[0].text.strip())
    print(cells[1].text.strip())
    print(cells[2].text.strip())
    print(cells[3].text.strip())
    print(cells[4].text.strip())

Holsteiner Chaussee 34
Lieselotte Brandt
Hier wohnteLieselotte Brandt Jg. 1936 eingewiesen 1943 Alsterdorfer Anstalten ‚verlegt‘ 16.8.1943 Am Spiegelgrund ‚Kinderfachabteilung‘ ermordet 17.1.1944

Eintrag auf stolpersteine-hamburg.de
Hornackredder 2
Heinrich Schröder
Hier wohnte Heinrich Schröder Jg. 1896 Schutzhaft 15.12.1944 KZ Fuhlsbüttel ermordet April 1945 Neuengamme

Eintrag auf stolpersteine-hamburg.de
Kieler Straße 594
Max Sommerfeld
Hier wohnte Max Sommerfeld Jg. 1897 KZ Fuhlsbüttel 1942 deportiert 1942 Mauthausen ermordet 15.8.1942

Eintrag auf stolpersteine-hamburg.deEin weiterer Stolperstein befindet sich in Hamburg-Hamm.
Schnackenburgallee 177–179
Anna Podolitsch geb. Antonenko
Hier interniert Lager ‚Russmann‘ Anna Podolitsch geb. Antonenko Jg. 1923 Ukraine Zwangsarbeit Butella WerkSchicksal unbekannt

Eintrag auf stolpersteine-hamburg.de
Schnackenburgallee 177–179
Knabe Podolitsch
Knabe Podolitsch geb. 1.5.1945 tot 6.5.1945

Eintrag auf stolpersteine-hamburg.de
Wiesenacke

In [433]:
## First Person should be Liselotte Brandt 

districts_eimsbüttel

districtlist, addresslist, namelist, inscriptionlist, picturelist, notelist = [],[],[],[],[],[]

count = 0

for table in table_list: 
    count = count + 1

    for row in table.select('tr')[1:]:
        cells = row.select("td")  
        addresslist.append(cells[0].text.strip())
        namelist.append(cells[1].text.strip())
        inscriptionlist.append(cells[2].text.strip())
        picturelist.append(cells[3].text.strip())
        notelist.append(cells[4].text.strip())
        districtlist.append(count)   


In [434]:
df = pd.DataFrame({'city' : 'Hamburg', 'borough' : 'Eimsbüttel', 'district' : districtlist, 'address': addresslist, 'name': namelist, 'inscription': inscriptionlist, 'picture' : picturelist, 'note' : notelist})
df

,city,borough,district,address,name,inscription,picture,note
0,Hamburg,Eimsbüttel,1,Holsteiner Chaussee 34,Lieselotte Brandt,Hier wohnteLieselotte Brandt Jg. 1936 eingewie...,,Eintrag auf stolpersteine-hamburg.de
1,Hamburg,Eimsbüttel,1,Hornackredder 2,Heinrich Schröder,Hier wohnte Heinrich Schröder Jg. 1896 Schutzh...,,Eintrag auf stolpersteine-hamburg.de
2,Hamburg,Eimsbüttel,1,Kieler Straße 594,Max Sommerfeld,Hier wohnte Max Sommerfeld Jg. 1897 KZ Fuhlsbü...,,Eintrag auf stolpersteine-hamburg.deEin weiter...
3,Hamburg,Eimsbüttel,1,Schnackenburgallee 177–179,Anna Podolitsch geb. Antonenko,Hier interniert Lager ‚Russmann‘ Anna Podolits...,,Eintrag auf stolpersteine-hamburg.de
4,Hamburg,Eimsbüttel,1,Schnackenburgallee 177–179,Knabe Podolitsch,Knabe Podolitsch geb. 1.5.1945 tot 6.5.1945,,Eintrag auf stolpersteine-hamburg.de
...,...,...,...,...,...,...,...,...
2917,Hamburg,Eimsbüttel,11,Försterweg 43,Ilse Lazarus,Hier wohnte Ilse Lazarus Jg. 1926 deportiert 1...,,Eintrag auf stolpersteine-hamburg.de
2918,Hamburg,Eimsbüttel,11,Steenwisch 70,Max Abraham Lichtenstein,Hier wohnte Max Abraham Lichtenstein Jg. 1874 ...,,Eintrag auf stolpersteine-hamburg.de
2919,Hamburg,Eimsbüttel,11,Steenwisch 70,Rebecka Lichtenstein geb. Salomon,Hier wohnte Rebecka Lichtenstein geb. Salomon ...,,Eintrag auf stolpersteine-hamburg.de
2920,Hamburg,Eimsbüttel,11,Vehrenkampstraße 15,Walter Friedrich,Hier wohnte Walter Friedrich Jg. 1908 eingewie...,,Eintrag auf stolpersteine-hamburg.de


In [435]:
from urllib.parse import urlparse, urlunparse

Geohack = []

for table in table_list: 
    for row in table.select('tr')[1:]:
        cells = []
        geo = []
        for cell in row.select("a"):
            cells.append(cell["href"])  
        for cell in cells:
            parsed_url = urlparse(cell)[4]
            geo.append(parsed_url)  
        for g in geo:
            if "pagename" in g:
                Geohack.append(g)
                
Geohack[:2]

['pagename=Liste_der_Stolpersteine_in_Hamburg-Eidelstedt&language=de&params=53.611230_N_9.905361_E_region:DE-HH_type:landmark&title=Lieselotte+Brandt',
 'pagename=Liste_der_Stolpersteine_in_Hamburg-Eidelstedt&language=de&params=53.611128_N_9.9009_E_region:DE-HH_type:landmark&title=Heinrich+Schr%C3%B6der']

In [436]:
df['Geohack'] = Geohack
params = df['Geohack'].str.extract("params=(.*)region", expand=False)
df['lat'] = params.str.extract("(.*)_N", expand=False)
df['lon'] = params.str.extract("N_(.*)_E", expand=False)

df['district'] = df['Geohack'].str.extract("pagename=Liste_der_Stolpersteine_in_Hamburg-(.*)&language", expand=False)

df

,city,borough,district,address,name,inscription,picture,note,Geohack,lat,lon
0,Hamburg,Eimsbüttel,Eidelstedt,Holsteiner Chaussee 34,Lieselotte Brandt,Hier wohnteLieselotte Brandt Jg. 1936 eingewie...,,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-Ei...,53.611230,9.905361
1,Hamburg,Eimsbüttel,Eidelstedt,Hornackredder 2,Heinrich Schröder,Hier wohnte Heinrich Schröder Jg. 1896 Schutzh...,,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-Ei...,53.611128,9.9009
2,Hamburg,Eimsbüttel,Eidelstedt,Kieler Straße 594,Max Sommerfeld,Hier wohnte Max Sommerfeld Jg. 1897 KZ Fuhlsbü...,,Eintrag auf stolpersteine-hamburg.deEin weiter...,pagename=Liste_der_Stolpersteine_in_Hamburg-Ei...,53.6012216,9.9147426
3,Hamburg,Eimsbüttel,Eidelstedt,Schnackenburgallee 177–179,Anna Podolitsch geb. Antonenko,Hier interniert Lager ‚Russmann‘ Anna Podolits...,,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-Ei...,53.5927123,9.90396557
4,Hamburg,Eimsbüttel,Eidelstedt,Schnackenburgallee 177–179,Knabe Podolitsch,Knabe Podolitsch geb. 1.5.1945 tot 6.5.1945,,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-Ei...,53.5927123,9.90396557
...,...,...,...,...,...,...,...,...,...,...,...
2917,Hamburg,Eimsbüttel,Stellingen,Försterweg 43,Ilse Lazarus,Hier wohnte Ilse Lazarus Jg. 1926 deportiert 1...,,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-St...,53.5785686,9.9304127
2918,Hamburg,Eimsbüttel,Stellingen,Steenwisch 70,Max Abraham Lichtenstein,Hier wohnte Max Abraham Lichtenstein Jg. 1874 ...,,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-St...,53.5828618,9.9449348
2919,Hamburg,Eimsbüttel,Stellingen,Steenwisch 70,Rebecka Lichtenstein geb. Salomon,Hier wohnte Rebecka Lichtenstein geb. Salomon ...,,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-St...,53.5828669,9.9449883
2920,Hamburg,Eimsbüttel,Stellingen,Vehrenkampstraße 15,Walter Friedrich,Hier wohnte Walter Friedrich Jg. 1908 eingewie...,,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-St...,53.5832168,9.9371286


In [437]:
df.to_csv('stolpersteine_eimsbüttel.csv', index = False)

## (4) Hamburg-Mitte – Problems with pictures and notes

In [438]:
districts_hamburg_mitte = []

for cell in hamburg_mitte.select("a"):
    district = cell.text.strip()
    url = "https://de.wikipedia.org" + cell["href"]          
    districts_hamburg_mitte.append({
        'Bezirk' : 'Hamburg-Mitte',
        'Stadtteil' : district,
        'URL' : url
    })    
    
districts_hamburg_mitte

[{'Bezirk': 'Hamburg-Mitte',
  'Stadtteil': 'Billbrook',
  'URL': 'https://de.wikipedia.org/wiki/Liste_der_Stolpersteine_in_Hamburg-Billbrook'},
 {'Bezirk': 'Hamburg-Mitte',
  'Stadtteil': 'Billstedt',
  'URL': 'https://de.wikipedia.org/wiki/Liste_der_Stolpersteine_in_Hamburg-Billstedt'},
 {'Bezirk': 'Hamburg-Mitte',
  'Stadtteil': 'Borgfelde',
  'URL': 'https://de.wikipedia.org/wiki/Liste_der_Stolpersteine_in_Hamburg-Borgfelde'},
 {'Bezirk': 'Hamburg-Mitte',
  'Stadtteil': 'Finkenwerder',
  'URL': 'https://de.wikipedia.org/wiki/Liste_der_Stolpersteine_in_Hamburg-Finkenwerder'},
 {'Bezirk': 'Hamburg-Mitte',
  'Stadtteil': 'Hamburg-Altstadt',
  'URL': 'https://de.wikipedia.org/wiki/Liste_der_Stolpersteine_in_Hamburg-Altstadt'},
 {'Bezirk': 'Hamburg-Mitte',
  'Stadtteil': 'Hamm',
  'URL': 'https://de.wikipedia.org/wiki/Liste_der_Stolpersteine_in_Hamburg-Hamm'},
 {'Bezirk': 'Hamburg-Mitte',
  'Stadtteil': 'Hammerbrook',
  'URL': 'https://de.wikipedia.org/wiki/Liste_der_Stolpersteine_in_Ha

In [439]:
# In Hamburg-Mitte you have to fix the <br> in the html

districts_hamburg_mitte

table_list = []
for d in districts_hamburg_mitte:
    url = d['URL'] 
    html = requests.get(url).text
    html = html.replace("<br /><b>", " ")
    html = html.replace("</b><br />", " ")
    html = html.replace("<br />", " ")
    soup = BeautifulSoup(html)
    d['Table'] = soup.select("table.wikitable")[0]
    d_table = d['Table']
    table_list.append(d_table)

len(table_list)

15

In [440]:
# Check if all the columns are the same

columns_hamburg_mitte = []
for table in table_list: 
    columns_per_table = []
    for row in table.select('tr')[1:]:
        columns = []
        header_el = d_table.select("tr")[0]
        for cell in header_el.select("th"):
            columns.append(cell.text.strip())
    columns_per_table.append(columns)   
    columns_hamburg_mitte.append(columns_per_table)
columns_hamburg_mitte

[[['Adresse', 'Person(en)', 'Inschrift', 'Bilder', 'Anmerkung']],
 [['Adresse', 'Person(en)', 'Inschrift', 'Bilder', 'Anmerkung']],
 [['Adresse', 'Person(en)', 'Inschrift', 'Bilder', 'Anmerkung']],
 [['Adresse', 'Person(en)', 'Inschrift', 'Bilder', 'Anmerkung']],
 [['Adresse', 'Person(en)', 'Inschrift', 'Bilder', 'Anmerkung']],
 [['Adresse', 'Person(en)', 'Inschrift', 'Bilder', 'Anmerkung']],
 [['Adresse', 'Person(en)', 'Inschrift', 'Bilder', 'Anmerkung']],
 [['Adresse', 'Person(en)', 'Inschrift', 'Bilder', 'Anmerkung']],
 [['Adresse', 'Person(en)', 'Inschrift', 'Bilder', 'Anmerkung']],
 [['Adresse', 'Person(en)', 'Inschrift', 'Bilder', 'Anmerkung']],
 [['Adresse', 'Person(en)', 'Inschrift', 'Bilder', 'Anmerkung']],
 [['Adresse', 'Person(en)', 'Inschrift', 'Bilder', 'Anmerkung']],
 [['Adresse', 'Person(en)', 'Inschrift', 'Bilder', 'Anmerkung']],
 [['Adresse', 'Person(en)', 'Inschrift', 'Bilder', 'Anmerkung']],
 [['Adresse', 'Person(en)', 'Inschrift', 'Bilder', 'Anmerkung']]]

In [441]:
for row in table_list[0].select('tr')[1:]:
    cells = row.select("td")   
    print(cells[0].text.strip())
    print(cells[1].text.strip())
    print(cells[2].text.strip())
    print(cells[3].text.strip())
    print(cells[4].text.strip())

Billbrookdeich 152
Ljuba Androssowa
Hier arbeitete Ljuba Androssowa Jg. 1913 Russland Zwangsarbeit 1943 eingewiesen 22.6.1943 Heilanstalt Langenhorn 'verlegt' 19.8.1943 Heilanstalt Meseritz-Obrawalde ermordet 25.11.1943

Eintrag auf stolpersteine-hamburg.de
Billbrookdeich 152
Clara Tuch
Hier wohnte und arbeitete Clara Tuch geb. Levie Jg. 1875 deportiert 1942 Theresienstadt Treblinka ermordet

Eintrag auf stolpersteine-hamburg.de Ein weiterer Stolperstein befindet sich in Hamburg-Volksdorf.
Billbrookdeich 152
Theodor Tuch
Hier wohnte und arbeitete Dr. Theodor Tuch Jg. 1865 deportiert 1942 Theresienstadt Treblinka ermordet

Eintrag auf stolpersteine-hamburg.de Ein weiterer Stolperstein befindet sich in Hamburg-Volksdorf.


In [442]:
## problems with picturelist and notelist - why?

districts_hamburg_mitte

districtlist, addresslist, namelist, inscriptionlist, picturelist, notelist = [],[],[],[],[],[]

count = 0

for table in table_list: 
    count = count + 1

    for row in table.select('tr')[1:]:
        cells = row.select("td")  
        addresslist.append(cells[0].text.strip())
        namelist.append(cells[1].text.strip())
        inscriptionlist.append(cells[2].text.strip())
#        picturelist.append(cells[3].text.strip())
#        notelist.append(cells[4].text.strip())
        districtlist.append(count)       

In [443]:
df = pd.DataFrame({'city' : 'Hamburg', 'borough' : 'Hamburg-Mitte', 'district' : districtlist, 'address': addresslist, 'name': namelist, 'inscription': inscriptionlist, 'picture' : "Fix picturelist", 'note' : "Fix notelist"})
df

,city,borough,district,address,name,inscription,picture,note
0,Hamburg,Hamburg-Mitte,1,Billbrookdeich 152,Ljuba Androssowa,Hier arbeitete Ljuba Androssowa Jg. 1913 Russl...,Fix picturelist,Fix notelist
1,Hamburg,Hamburg-Mitte,1,Billbrookdeich 152,Clara Tuch,Hier wohnte und arbeitete Clara Tuch geb. Levi...,Fix picturelist,Fix notelist
2,Hamburg,Hamburg-Mitte,1,Billbrookdeich 152,Theodor Tuch,Hier wohnte und arbeitete Dr. Theodor Tuch Jg....,Fix picturelist,Fix notelist
3,Hamburg,Hamburg-Mitte,2,Billstedter Hauptstraße 8,Paula Laser,Hier wohnte Paula Laser geb. Schleimer Jg. ...,Fix picturelist,Fix notelist
4,Hamburg,Hamburg-Mitte,2,Billstedter Hauptstraße 8,Simon Laser,Hier wohnte Simon Laser Jg. 1882 deportiert...,Fix picturelist,Fix notelist
...,...,...,...,...,...,...,...,...
1454,Hamburg,Hamburg-Mitte,15,Vogelhüttendeich 34,Fanny Borower geb. Schwarz,Hier wohnte Fanny Borower geb. Schwarz Jg. 187...,Fix picturelist,Fix notelist
1455,Hamburg,Hamburg-Mitte,15,Vogelhüttendeich 34,Wolf Borower,Hier wohnte Wolf Borower Jg. 1870 deportiert 1...,Fix picturelist,Fix notelist
1456,Hamburg,Hamburg-Mitte,15,Vogelhüttendeich 40,Clara Cohn geb. Laser,Hier wohnte Clara Cohn geb. Laser Jg. 1888 dep...,Fix picturelist,Fix notelist
1457,Hamburg,Hamburg-Mitte,15,Vogelhüttendeich 40,Adolf Laser,Hier wohnte Adolf Laser Jg. 1918 1941 KZ Fuhls...,Fix picturelist,Fix notelist


In [444]:
from urllib.parse import urlparse, urlunparse

Geohack = []

for table in table_list: 
    for row in table.select('tr')[1:]:
        cells = []
        geo = []
        for cell in row.select("a"):
            cells.append(cell["href"])  
        for cell in cells:
            parsed_url = urlparse(cell)[4]
            geo.append(parsed_url)  
        for g in geo:
            if "pagename" in g:
                Geohack.append(g)
else:
    Geohack.append('Nan') 
                
Geohack[:10]

['pagename=Liste_der_Stolpersteine_in_Hamburg-Billbrook&language=de&params=53.536138_N_10.096823_E_region:DE-HH_type:landmark&title=Ljuba+Androssowa',
 'pagename=Liste_der_Stolpersteine_in_Hamburg-Billbrook&language=de&params=53.536138_N_10.096823_E_region:DE-HH_type:landmark&title=Clara+Tuch',
 'pagename=Liste_der_Stolpersteine_in_Hamburg-Billbrook&language=de&params=53.536138_N_10.096823_E_region:DE-HH_type:landmark&title=Dr.+Theodor+Tuch',
 'pagename=Liste_der_Stolpersteine_in_Hamburg-Billstedt&language=de&params=53.5420248_N_10.0966851_E_region:DE-HH_type:landmark&title=Paula+Laser',
 'pagename=Liste_der_Stolpersteine_in_Hamburg-Billstedt&language=de&params=53.5420309_N_10.0966983_E_region:DE-HH_type:landmark&title=Simon+Laser',
 'pagename=Liste_der_Stolpersteine_in_Hamburg-Billstedt&language=de&params=53.5397644_N_10.1022077_E_region:DE-HH_type:landmark&title=David+Isenberg',
 'pagename=Liste_der_Stolpersteine_in_Hamburg-Billstedt&language=de&params=53.5397768_N_10.1022370_E_regio

In [445]:
df['Geohack'] = Geohack

params = df['Geohack'].str.extract("params=(.*)region", expand=False)
df['lat'] = params.str.extract("(.*)_N", expand=False)
df['lon'] = params.str.extract("N_(.*)_E", expand=False)

df['district'] = df['Geohack'].str.extract("pagename=Liste_der_Stolpersteine_in_Hamburg-(.*)&language", expand=False)

df

,city,borough,district,address,name,inscription,picture,note,Geohack,lat,lon
0,Hamburg,Hamburg-Mitte,Billbrook,Billbrookdeich 152,Ljuba Androssowa,Hier arbeitete Ljuba Androssowa Jg. 1913 Russl...,Fix picturelist,Fix notelist,pagename=Liste_der_Stolpersteine_in_Hamburg-Bi...,53.536138,10.096823
1,Hamburg,Hamburg-Mitte,Billbrook,Billbrookdeich 152,Clara Tuch,Hier wohnte und arbeitete Clara Tuch geb. Levi...,Fix picturelist,Fix notelist,pagename=Liste_der_Stolpersteine_in_Hamburg-Bi...,53.536138,10.096823
2,Hamburg,Hamburg-Mitte,Billbrook,Billbrookdeich 152,Theodor Tuch,Hier wohnte und arbeitete Dr. Theodor Tuch Jg....,Fix picturelist,Fix notelist,pagename=Liste_der_Stolpersteine_in_Hamburg-Bi...,53.536138,10.096823
3,Hamburg,Hamburg-Mitte,Billstedt,Billstedter Hauptstraße 8,Paula Laser,Hier wohnte Paula Laser geb. Schleimer Jg. ...,Fix picturelist,Fix notelist,pagename=Liste_der_Stolpersteine_in_Hamburg-Bi...,53.5420248,10.0966851
4,Hamburg,Hamburg-Mitte,Billstedt,Billstedter Hauptstraße 8,Simon Laser,Hier wohnte Simon Laser Jg. 1882 deportiert...,Fix picturelist,Fix notelist,pagename=Liste_der_Stolpersteine_in_Hamburg-Bi...,53.5420309,10.0966983
...,...,...,...,...,...,...,...,...,...,...,...
1454,Hamburg,Hamburg-Mitte,Wilhelmsburg,Vogelhüttendeich 34,Fanny Borower geb. Schwarz,Hier wohnte Fanny Borower geb. Schwarz Jg. 187...,Fix picturelist,Fix notelist,pagename=Liste_der_Stolpersteine_in_Hamburg-Wi...,53.5181911,9.9845291
1455,Hamburg,Hamburg-Mitte,Wilhelmsburg,Vogelhüttendeich 34,Wolf Borower,Hier wohnte Wolf Borower Jg. 1870 deportiert 1...,Fix picturelist,Fix notelist,pagename=Liste_der_Stolpersteine_in_Hamburg-Wi...,53.5180032,9.9852649
1456,Hamburg,Hamburg-Mitte,Wilhelmsburg,Vogelhüttendeich 40,Clara Cohn geb. Laser,Hier wohnte Clara Cohn geb. Laser Jg. 1888 dep...,Fix picturelist,Fix notelist,pagename=Liste_der_Stolpersteine_in_Hamburg-Wi...,53.5180168,9.9851882
1457,Hamburg,Hamburg-Mitte,Wilhelmsburg,Vogelhüttendeich 40,Adolf Laser,Hier wohnte Adolf Laser Jg. 1918 1941 KZ Fuhls...,Fix picturelist,Fix notelist,pagename=Liste_der_Stolpersteine_in_Hamburg-Wi...,53.5180102,9.9852262


In [446]:
df.to_csv('stolpersteine_hamburg_mitte.csv', index = False)

## (5) Hamburg-Nord - Looks complete

In [447]:
districts_hamburg_nord = []

for cell in hamburg_nord.select("a"):
    district = cell.text.strip()
    url = "https://de.wikipedia.org" + cell["href"]          
    districts_hamburg_nord.append({
        'Bezirk' : 'Hamburg-Nord',
        'Stadtteil' : district,
        'URL' : url
    })    

In [448]:
districts_hamburg_nord

table_list = []
for d in districts_hamburg_nord:
    url = d['URL'] 
    html = requests.get(url).text
    soup = BeautifulSoup(html)
    d['Table'] = soup.select("table.wikitable")[0]
    d_table = d['Table']
    table_list.append(d_table)

#table_list[1]
len(table_list)

13

In [449]:
districts_hamburg_nord

districtlist, addresslist, namelist, inscriptionlist, picturelist, notelist = [],[],[],[],[],[]

count = 0

for table in table_list: 
    count = count + 1

    for row in table.select('tr')[1:]:
        cells = row.select("td")  
        addresslist.append(cells[0].text.strip())
        namelist.append(cells[1].text.strip())
        inscriptionlist.append(cells[2].text.strip())
        picturelist.append(cells[3].text.strip())
        notelist.append(cells[4].text.strip())
        districtlist.append(count)      

In [450]:
df = pd.DataFrame({'city': 'Hamburg', 'borough' : 'Hamburg-Nord', 'district' : districtlist, 'address': addresslist, 'name': namelist, 'inscription': inscriptionlist, 'picture' : picturelist, 'note' : notelist})
df

,city,borough,district,address,name,inscription,picture,note
0,Hamburg,Hamburg-Nord,1,Feuerbergstraße 43,Karl-Heinz Dohrmann,Karl-HeinzDohrmannJg. 1937untergebracht 1940Ki...,Stolperstein für Karl-Heinz Dohrmann,Eintrag auf stolpersteine-hamburg.de
1,Hamburg,Hamburg-Nord,1,Feuerbergstraße 43,Anna-Maria Tomassetti,Anna-MariaTomassettigeb. 15.3.1945untergebrach...,Stolperstein für Anna-Maria Tomassetti,Eintrag auf stolpersteine-hamburg.de
2,Hamburg,Hamburg-Nord,1,Heilholtkamp 21,Käthe Consentius,Hier wohnteKäthe ConsentiusJg. 1912eingewiesen...,Stolperstein für Käthe Consentius,Eintrag auf stolpersteine-hamburg.de
3,Hamburg,Hamburg-Nord,1,Hindenburgstraße 111,Hedwig Wohlwill geb. Dehn,Hier wohnteHedwig Wohlwillgeb. DehnJg. 1877dep...,Stolperstein für Hedwig Wohlwill geb. Dehn,Eintrag auf stolpersteine-hamburg.de
4,Hamburg,Hamburg-Nord,1,Hindenburgstraße 111,Heinrich Wohlwill,Hier wohnteDr. HeinrichWohlwillJg. 1874deporti...,Stolperstein für Dr. Heinrich Wohlwill,Eintrag auf stolpersteine-hamburg.deWeitere St...
...,...,...,...,...,...,...,...,...
1507,Hamburg,Hamburg-Nord,13,Zesenstraße 11,Melanie Peters geb. Waschinsky,Hier wohnte Melanie Peters geb. Waschinsky Jg....,,Eintrag auf stolpersteine-hamburg.de
1508,Hamburg,Hamburg-Nord,13,Zesenstraße 11,Eveline Waschinsky,Hier wohnte Eveline Waschinsky Jg. 1898 deport...,,Eintrag auf stolpersteine-hamburg.de
1509,Hamburg,Hamburg-Nord,13,Zesenstraße 13,Susanne Wolf geb. Vogel,Hier wohnte Susanne Wolf geb. Vogel Jg. 1866 g...,,Eintrag auf stolpersteine-hamburg.de
1510,Hamburg,Hamburg-Nord,13,Zesenstraße 15,Herbert Epstein,Hier wohnte Herbert Epstein Jg. 1892 deportier...,,Eintrag auf stolpersteine-hamburg.de


In [451]:
from urllib.parse import urlparse, urlunparse

Geohack = []

for table in table_list: 
    for row in table.select('tr')[1:]:
        cells = []
        geo = []
        for cell in row.select("a"):
            cells.append(cell["href"])  
        for cell in cells:
            parsed_url = urlparse(cell)[4]
            geo.append(parsed_url)  
        for g in geo:
            if "pagename" in g:
                Geohack.append(g)
                
Geohack[:2]

['pagename=Liste_der_Stolpersteine_in_Hamburg-Alsterdorf&language=de&params=53.613062_N_10.029798_E_region:DE-HH_type:landmark&title=Karl-Heinz+Dohrmann',
 'pagename=Liste_der_Stolpersteine_in_Hamburg-Alsterdorf&language=de&params=53.613062_N_10.029798_E_region:DE-HH_type:landmark&title=Anna-Maria+Tomassetti']

In [452]:
df['Geohack'] = Geohack

params = df['Geohack'].str.extract("params=(.*)region", expand=False)
df['lat'] = params.str.extract("(.*)_N", expand=False)
df['lon'] = params.str.extract("N_(.*)_E", expand=False)

df['district'] = df['Geohack'].str.extract("pagename=Liste_der_Stolpersteine_in_Hamburg-(.*)&language", expand=False)

df

,city,borough,district,address,name,inscription,picture,note,Geohack,lat,lon
0,Hamburg,Hamburg-Nord,Alsterdorf,Feuerbergstraße 43,Karl-Heinz Dohrmann,Karl-HeinzDohrmannJg. 1937untergebracht 1940Ki...,Stolperstein für Karl-Heinz Dohrmann,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-Al...,53.613062,10.029798
1,Hamburg,Hamburg-Nord,Alsterdorf,Feuerbergstraße 43,Anna-Maria Tomassetti,Anna-MariaTomassettigeb. 15.3.1945untergebrach...,Stolperstein für Anna-Maria Tomassetti,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-Al...,53.613062,10.029798
2,Hamburg,Hamburg-Nord,Alsterdorf,Heilholtkamp 21,Käthe Consentius,Hier wohnteKäthe ConsentiusJg. 1912eingewiesen...,Stolperstein für Käthe Consentius,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-Al...,53.610412,10.014295
3,Hamburg,Hamburg-Nord,Alsterdorf,Hindenburgstraße 111,Hedwig Wohlwill geb. Dehn,Hier wohnteHedwig Wohlwillgeb. DehnJg. 1877dep...,Stolperstein für Hedwig Wohlwill geb. Dehn,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-Al...,53.6108277,10.0102982
4,Hamburg,Hamburg-Nord,Alsterdorf,Hindenburgstraße 111,Heinrich Wohlwill,Hier wohnteDr. HeinrichWohlwillJg. 1874deporti...,Stolperstein für Dr. Heinrich Wohlwill,Eintrag auf stolpersteine-hamburg.deWeitere St...,pagename=Liste_der_Stolpersteine_in_Hamburg-Al...,53.6108134,10.0103049
...,...,...,...,...,...,...,...,...,...,...,...
1507,Hamburg,Hamburg-Nord,Winterhude,Zesenstraße 11,Melanie Peters geb. Waschinsky,Hier wohnte Melanie Peters geb. Waschinsky Jg....,,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-Wi...,53.5883892,10.0059338
1508,Hamburg,Hamburg-Nord,Winterhude,Zesenstraße 11,Eveline Waschinsky,Hier wohnte Eveline Waschinsky Jg. 1898 deport...,,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-Wi...,53.5883853,10.0059282
1509,Hamburg,Hamburg-Nord,Winterhude,Zesenstraße 13,Susanne Wolf geb. Vogel,Hier wohnte Susanne Wolf geb. Vogel Jg. 1866 g...,,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-Wi...,53.588491,10.006060
1510,Hamburg,Hamburg-Nord,Winterhude,Zesenstraße 15,Herbert Epstein,Hier wohnte Herbert Epstein Jg. 1892 deportier...,,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-Wi...,53.5886670,10.0062241


In [453]:
df.to_csv('stolpersteine_hamburg_nord.csv', index = False)

## (6) Harburg - Fix district Ei%C3%9Fendorf

In [464]:
districts_harburg = []

for cell in harburg.select("a"):
    district = cell.text.strip()
    url = "https://de.wikipedia.org" + cell["href"]          
    districts_harburg.append({
        'Bezirk' : 'Harburg',
        'Stadtteil' : district,
        'URL' : url,
    })    
districts_harburg

[{'Bezirk': 'Harburg',
  'Stadtteil': 'Altenwerder',
  'URL': 'https://de.wikipedia.org/wiki/Liste_der_Stolpersteine_in_Hamburg-Altenwerder'},
 {'Bezirk': 'Harburg',
  'Stadtteil': 'Eißendorf',
  'URL': 'https://de.wikipedia.org/wiki/Liste_der_Stolpersteine_in_Hamburg-Ei%C3%9Fendorf'},
 {'Bezirk': 'Harburg',
  'Stadtteil': 'Harburg',
  'URL': 'https://de.wikipedia.org/wiki/Liste_der_Stolpersteine_in_Hamburg-Harburg'},
 {'Bezirk': 'Harburg',
  'Stadtteil': 'Hausbruch',
  'URL': 'https://de.wikipedia.org/wiki/Liste_der_Stolpersteine_in_Hamburg-Hausbruch'},
 {'Bezirk': 'Harburg',
  'Stadtteil': 'Heimfeld',
  'URL': 'https://de.wikipedia.org/wiki/Liste_der_Stolpersteine_in_Hamburg-Heimfeld'},
 {'Bezirk': 'Harburg',
  'Stadtteil': 'Marmstorf',
  'URL': 'https://de.wikipedia.org/wiki/Liste_der_Stolpersteine_in_Hamburg-Marmstorf'},
 {'Bezirk': 'Harburg',
  'Stadtteil': 'Neuenfelde',
  'URL': 'https://de.wikipedia.org/wiki/Liste_der_Stolpersteine_in_Hamburg-Neuenfelde'},
 {'Bezirk': 'Harburg',

In [465]:
districts_harburg

table_list = []
for d in districts_harburg:
    url = d['URL'] 
    html = requests.get(url).text
    html = html.replace("<br /><b>", " ")
    html = html.replace("</b><br />", " ")
    html = html.replace("<br />", " ")
    soup = BeautifulSoup(html)
    d['Table'] = soup.select("table.wikitable")[0]
    d_table = d['Table']
    table_list.append(d_table)

len(table_list)

11

In [466]:
# Check if all the columns are the same

columns_harburg = []
for table in table_list: 
    columns_per_table = []
    for row in table.select('tr')[1:]:
        columns = []
        header_el = d_table.select("tr")[0]
        for cell in header_el.select("th"):
            columns.append(cell.text.strip())
    columns_per_table.append(columns)   
    columns_harburg.append(columns_per_table)
columns_harburg

[[['Adresse', 'Person(en)', 'Inschrift', 'Bilder', 'Anmerkung']],
 [['Adresse', 'Person(en)', 'Inschrift', 'Bilder', 'Anmerkung']],
 [['Adresse', 'Person(en)', 'Inschrift', 'Bilder', 'Anmerkung']],
 [['Adresse', 'Person(en)', 'Inschrift', 'Bilder', 'Anmerkung']],
 [['Adresse', 'Person(en)', 'Inschrift', 'Bilder', 'Anmerkung']],
 [['Adresse', 'Person(en)', 'Inschrift', 'Bilder', 'Anmerkung']],
 [['Adresse', 'Person(en)', 'Inschrift', 'Bilder', 'Anmerkung']],
 [['Adresse', 'Person(en)', 'Inschrift', 'Bilder', 'Anmerkung']],
 [['Adresse', 'Person(en)', 'Inschrift', 'Bilder', 'Anmerkung']],
 [['Adresse', 'Person(en)', 'Inschrift', 'Bilder', 'Anmerkung']],
 [['Adresse', 'Person(en)', 'Inschrift', 'Bilder', 'Anmerkung']]]

In [467]:

districts_harburg

districtlist, addresslist, namelist, inscriptionlist, picturelist, notelist = [],[],[],[],[],[]

count = 0

for table in table_list: 
    count = count + 1

    for row in table.select('tr')[1:]:
        cells = row.select("td")  
        addresslist.append(cells[0].text.strip())
        namelist.append(cells[1].text.strip())
        inscriptionlist.append(cells[2].text.strip())
        picturelist.append(cells[3].text.strip())
        notelist.append(cells[4].text.strip())
        districtlist.append(count)   

In [468]:
df = pd.DataFrame({'city': 'Hamburg', 'borough' : 'Harburg', 'district' : districtlist, 'address': addresslist, 'name': namelist, 'inscription': inscriptionlist, 'picture' : picturelist, 'note' : notelist})
df

,city,borough,district,address,name,inscription,picture,note
0,Hamburg,Harburg,1,Altenwerder Kirchweg 3a,Rudolf Krukenbaum,Hier wohnte Rudolf Krukenbaum Jg. 1904 verhaft...,Stolperstein für Rudolf Krukenbaum,Eintrag auf stolpersteine-hamburg.de
1,Hamburg,Harburg,2,Bremer Straße 155,Günther Driewer,Hier wohnte Günther Driewer Jg. 1934 eingewies...,,Eintrag auf stolpersteine-hamburg.de
2,Hamburg,Harburg,2,Ehestorfer Weg 99,Robert Homeyer,Hier wohnte Robert Homeyer Jg. 1908 im Widerst...,,Eintrag auf stolpersteine-hamburg.de
3,Hamburg,Harburg,2,Eißendorfer Straße 62,Heinz Pabst,Hier wohnte Heinz Papst[A 1] Jg. 1933 einge...,,Eintrag auf stolpersteine-hamburg.de
4,Hamburg,Harburg,2,Eißendorfer Straße 88,Albert Voigts,Hier wohnte Albert Voigts Jg. 1904 im Widersta...,,Eintrag auf stolpersteine-hamburg.de
...,...,...,...,...,...,...,...,...
252,Hamburg,Harburg,11,Reeseberg 109,Martha Bedey geb. Metschies,Hier wohnte Martha Bedey geb. Metschies Jg....,,Eintrag auf stolpersteine-hamburg.de
253,Hamburg,Harburg,11,Rönneburger Straße 32,Margarete Heinsen geb. Lampert,Hier wohnte Margarete Heinsen geb. Lampert ...,,Eintrag auf stolpersteine-hamburg.de
254,Hamburg,Harburg,11,Winsener Straße 19,Günther Fitschen,Hier wohnte Günther Fitschen Jg. 1933 eingewie...,,Eintrag auf stolpersteine-hamburg.de
255,Hamburg,Harburg,11,Winsener Straße 30,Alfons Teschke,Hier wohnte Alfons Teschke Jg. 1898 Spanien...,,Eintrag auf stolpersteine-hamburg.de


In [469]:
from urllib.parse import urlparse, urlunparse

Geohack = []

for table in table_list: 
    for row in table.select('tr')[1:]:
        cells = []
        geo = []
        for cell in row.select("a"):
            cells.append(cell["href"])  
        for cell in cells:
            parsed_url = urlparse(cell)[4]
            geo.append(parsed_url)  
        for g in geo:
            if "pagename" in g:
                Geohack.append(g)

                
Geohack[:3]

['pagename=Liste_der_Stolpersteine_in_Hamburg-Altenwerder&language=de&params=53.5063495_N_9.9176987_E_region:DE-HH_type:landmark&title=Rudolf+Krukenbaum',
 'pagename=Liste_der_Stolpersteine_in_Hamburg-Ei%C3%9Fendorf&language=de&params=53.450147_N_9.967883_E_region:DE-HH_type:landmark&title=G%C3%BCnther+Driewer',
 'pagename=Liste_der_Stolpersteine_in_Hamburg-Ei%C3%9Fendorf&language=de&params=53.455299_N_9.941738_E_region:DE-HH_type:landmark&title=Robert+Homeyer']

In [470]:
## Fix district Ei%C3%9Fendorf

df['Geohack'] = Geohack

params = df['Geohack'].str.extract("params=(.*)region", expand=False)
df['lat'] = params.str.extract("(.*)_N", expand=False)
df['lon'] = params.str.extract("N_(.*)_E", expand=False)

df['district'] = df['Geohack'].str.extract("pagename=Liste_der_Stolpersteine_in_Hamburg-(.*)&language", expand=False)
    
df

,city,borough,district,address,name,inscription,picture,note,Geohack,lat,lon
0,Hamburg,Harburg,Altenwerder,Altenwerder Kirchweg 3a,Rudolf Krukenbaum,Hier wohnte Rudolf Krukenbaum Jg. 1904 verhaft...,Stolperstein für Rudolf Krukenbaum,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-Al...,53.5063495,9.9176987
1,Hamburg,Harburg,Ei%C3%9Fendorf,Bremer Straße 155,Günther Driewer,Hier wohnte Günther Driewer Jg. 1934 eingewies...,,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-Ei...,53.450147,9.967883
2,Hamburg,Harburg,Ei%C3%9Fendorf,Ehestorfer Weg 99,Robert Homeyer,Hier wohnte Robert Homeyer Jg. 1908 im Widerst...,,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-Ei...,53.455299,9.941738
3,Hamburg,Harburg,Ei%C3%9Fendorf,Eißendorfer Straße 62,Heinz Pabst,Hier wohnte Heinz Papst[A 1] Jg. 1933 einge...,,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-Ei...,53.458277,9.967645
4,Hamburg,Harburg,Ei%C3%9Fendorf,Eißendorfer Straße 88,Albert Voigts,Hier wohnte Albert Voigts Jg. 1904 im Widersta...,,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-Ei...,53.4577,9.96466
...,...,...,...,...,...,...,...,...,...,...,...
252,Hamburg,Harburg,Wilstorf,Reeseberg 109,Martha Bedey geb. Metschies,Hier wohnte Martha Bedey geb. Metschies Jg....,,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-Wi...,53.444450,9.998382
253,Hamburg,Harburg,Wilstorf,Rönneburger Straße 32,Margarete Heinsen geb. Lampert,Hier wohnte Margarete Heinsen geb. Lampert ...,,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-Wi...,53.440817,9.993406
254,Hamburg,Harburg,Wilstorf,Winsener Straße 19,Günther Fitschen,Hier wohnte Günther Fitschen Jg. 1933 eingewie...,,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-Wi...,53.450683,9.987684
255,Hamburg,Harburg,Wilstorf,Winsener Straße 30,Alfons Teschke,Hier wohnte Alfons Teschke Jg. 1898 Spanien...,,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-Wi...,53.449162,9.987573


In [471]:
df.to_csv('stolpersteine_harburg.csv', index = False)

## (7) Wandsbek

In [472]:
districts_wandsbek = []

for cell in wandsbek.select("a"):
    district = cell.text.strip()
    url = "https://de.wikipedia.org" + cell["href"]          
    districts_wandsbek.append({
        'Bezirk' : 'Wandsbek',
        'Stadtteil' : district,
        'URL' : url,
    })    
districts_wandsbek   

[{'Bezirk': 'Wandsbek',
  'Stadtteil': 'Bergstedt',
  'URL': 'https://de.wikipedia.org/wiki/Liste_der_Stolpersteine_in_Hamburg-Bergstedt'},
 {'Bezirk': 'Wandsbek',
  'Stadtteil': 'Bramfeld',
  'URL': 'https://de.wikipedia.org/wiki/Liste_der_Stolpersteine_in_Hamburg-Bramfeld'},
 {'Bezirk': 'Wandsbek',
  'Stadtteil': 'Eilbek',
  'URL': 'https://de.wikipedia.org/wiki/Liste_der_Stolpersteine_in_Hamburg-Eilbek'},
 {'Bezirk': 'Wandsbek',
  'Stadtteil': 'Farmsen-Berne',
  'URL': 'https://de.wikipedia.org/wiki/Liste_der_Stolpersteine_in_Hamburg-Farmsen-Berne'},
 {'Bezirk': 'Wandsbek',
  'Stadtteil': 'Hummelsbüttel',
  'URL': 'https://de.wikipedia.org/wiki/Liste_der_Stolpersteine_in_Hamburg-Hummelsb%C3%BCttel'},
 {'Bezirk': 'Wandsbek',
  'Stadtteil': 'Lehmsahl-Mellingstedt',
  'URL': 'https://de.wikipedia.org/wiki/Liste_der_Stolpersteine_in_Hamburg-Lemsahl-Mellingstedt'},
 {'Bezirk': 'Wandsbek',
  'Stadtteil': 'Marienthal',
  'URL': 'https://de.wikipedia.org/wiki/Liste_der_Stolpersteine_in_Hamb

In [473]:
districts_wandsbek

table_list = []
for d in districts_wandsbek:
    url = d['URL'] 
    html = requests.get(url).text
    html = html.replace("<br /><b>", " ")
    html = html.replace("</b><br />", " ")
    html = html.replace("<br />", " ")
    soup = BeautifulSoup(html)
    d['Table'] = soup.select("table.wikitable")[0]
    d_table = d['Table']
    table_list.append(d_table)

len(table_list)

15

In [474]:
# Check if all the columns are the same

columns_wandsbek = []
for table in table_list: 
    columns_per_table = []
    for row in table.select('tr')[1:]:
        columns = []
        header_el = d_table.select("tr")[0]
        for cell in header_el.select("th"):
            columns.append(cell.text.strip())
    columns_per_table.append(columns)   
    columns_wandsbek.append(columns_per_table)
columns_wandsbek

[[['Adresse', 'Person(en)', 'Inschrift', 'Bilder', 'Anmerkung']],
 [['Adresse', 'Person(en)', 'Inschrift', 'Bilder', 'Anmerkung']],
 [['Adresse', 'Person(en)', 'Inschrift', 'Bilder', 'Anmerkung']],
 [['Adresse', 'Person(en)', 'Inschrift', 'Bilder', 'Anmerkung']],
 [['Adresse', 'Person(en)', 'Inschrift', 'Bilder', 'Anmerkung']],
 [['Adresse', 'Person(en)', 'Inschrift', 'Bilder', 'Anmerkung']],
 [['Adresse', 'Person(en)', 'Inschrift', 'Bilder', 'Anmerkung']],
 [['Adresse', 'Person(en)', 'Inschrift', 'Bilder', 'Anmerkung']],
 [['Adresse', 'Person(en)', 'Inschrift', 'Bilder', 'Anmerkung']],
 [['Adresse', 'Person(en)', 'Inschrift', 'Bilder', 'Anmerkung']],
 [['Adresse', 'Person(en)', 'Inschrift', 'Bilder', 'Anmerkung']],
 [['Adresse', 'Person(en)', 'Inschrift', 'Bilder', 'Anmerkung']],
 [['Adresse', 'Person(en)', 'Inschrift', 'Bilder', 'Anmerkung']],
 [['Adresse', 'Person(en)', 'Inschrift', 'Bilder', 'Anmerkung']],
 [['Adresse', 'Person(en)', 'Inschrift', 'Bilder', 'Anmerkung']]]

In [475]:

districts_wandsbek

districtlist, addresslist, namelist, inscriptionlist, picturelist, notelist = [],[],[],[],[],[]

count = 0

for table in table_list: 
    count = count + 1

    for row in table.select('tr')[1:]:
        cells = row.select("td")  
        addresslist.append(cells[0].text.strip())
        namelist.append(cells[1].text.strip())
        inscriptionlist.append(cells[2].text.strip())
        picturelist.append(cells[3].text.strip())
        notelist.append(cells[4].text.strip())
        districtlist.append(count)   

In [476]:
df = pd.DataFrame({'city': 'Hamburg', 'borough' : 'Wandsbek', 'district' : districtlist, 'address': addresslist, 'name': namelist, 'inscription': inscriptionlist, 'picture' : picturelist, 'note' : notelist})
df

,city,borough,district,address,name,inscription,picture,note
0,Hamburg,Wandsbek,1,Baben de Möhl,Carl Schrader,Hier wohnte Carl Schrader Jg. 1907 im Widersta...,,Eintrag auf stolpersteine-hamburg.de
1,Hamburg,Wandsbek,1,Kastanienweg 5,Carl Sievers,Hier wohnte Carl Sievers Jg. 1921 eingewiesen ...,,Eintrag auf stolpersteine-hamburg.de
2,Hamburg,Wandsbek,2,Berner Chaussee 16,Wilhelm Czymmek,Hier wohnte Wilhelm Czymmek Jg. 1906 mehrfach ...,,Eintrag auf stolpersteine-hamburg.de
3,Hamburg,Wandsbek,2,Bramfelder Chaussee 269,Karl Lorenzen,Hier wohnte Dr. Karl Lorenzen Jg. 1889 verhaft...,,Eintrag auf stolpersteine-hamburg.de Stolperst...
4,Hamburg,Wandsbek,2,Bramfelder Chaussee 316,Leo Bornstein,Hier wohnte Leo Bornstein Jg. 1899 deportiert ...,,Eintrag auf stolpersteine-hamburg.de Stolperst...
...,...,...,...,...,...,...,...,...
257,Hamburg,Wandsbek,15,Hasselwisch 5,Georg Mewes,Hier wohnte Georg Mewes Jg. 1909 verhaftet 194...,,Eintrag auf stolpersteine-hamburg.de
258,Hamburg,Wandsbek,15,Lottbeker Weg 13,Dorothea Gatzenmeyer geb. Jürs,Hier wohnte Dorothea Gatzenmeyer geb. Jürs Jg....,,Eintrag auf stolpersteine-hamburg.de
259,Hamburg,Wandsbek,15,Lottbeker Weg 13,Louis Gatzenmeyer,Hier wohnte Louis Gatzenmeyer Jg. 1905 im Wide...,,Eintrag auf stolpersteine-hamburg.de
260,Hamburg,Wandsbek,15,Lottbeker Weg 24,Chaja Rywa Balck geb. Skop,Hier wohnte Chaja Rywa Balck geb. Skop Jg. 189...,,Eintrag auf stolpersteine-hamburg.de Stolperst...


In [477]:
from urllib.parse import urlparse, urlunparse

Geohack = []

for table in table_list: 
    for row in table.select('tr')[1:]:
        cells = []
        geo = []
        for cell in row.select("a"):
            cells.append(cell["href"])  
        for cell in cells:
            parsed_url = urlparse(cell)[4]
            geo.append(parsed_url)  
        for g in geo:
            if "pagename" in g:
                Geohack.append(g)
                
Geohack[:5]

['pagename=Liste_der_Stolpersteine_in_Hamburg-Bergstedt&language=de&params=53.673665_N_10.109742_E_region:DE-HH_type:landmark&title=Carl+Schrader',
 'pagename=Liste_der_Stolpersteine_in_Hamburg-Bergstedt&language=de&params=53.6610148_N_10.1295726_E_region:DE-HH_type:landmark&title=Carl+Sievers',
 'pagename=Liste_der_Stolpersteine_in_Hamburg-Bramfeld&language=de&params=53.618382_N_10.082536_E_region:DE-HH_type:landmark&title=Wilhelm+Czymmek',
 'pagename=Liste_der_Stolpersteine_in_Hamburg-Bramfeld&language=de&params=53.6155719_N_10.0763044_E_region:DE-HH_type:landmark&title=Dr.+Karl+Lorenzen',
 'pagename=Liste_der_Stolpersteine_in_Hamburg-Bramfeld&language=de&params=53.618377_N_10.079838_E_region:DE-HH_type:landmark&title=Leo+Bornstein']

In [478]:
# One value to much???

df['Geohack'] = "Fix Geohack"
df['lat'] = "Fix lat"
df['lon'] = "Fix lon"

#df['Geohack'] = Geohack

#params = df['Geohack'].str.extract("params=(.*)region", expand=False)
#df['lat'] = params.str.extract("(.*)_N", expand=False)
#df['lon'] = params.str.extract("N_(.*)_E", expand=False)

#df['district'] = df['Geohack'].str.extract("pagename=Liste_der_Stolpersteine_in_Hamburg-(.*)&language", expand=False)

#df

In [479]:
df.to_csv('stolpersteine_wandsbek.csv', index = False)

# Saving all dfs to df_hamburg

In [482]:
df1 = pd.read_csv('stolpersteine_altona.csv')
df2 = pd.read_csv('stolpersteine_bergedorf.csv')
df3 = pd.read_csv('stolpersteine_eimsbüttel.csv')
df4 = pd.read_csv('stolpersteine_hamburg_mitte.csv')
df5 = pd.read_csv('stolpersteine_hamburg_nord.csv')
df6 = pd.read_csv('stolpersteine_harburg.csv')
df7 = pd.read_csv('stolpersteine_wandsbek.csv')
df_hamburg = pd.concat([df1, df2, df3, df4, df5, df6, df7], sort=False)
df_hamburg

,city,borough,district,address,name,inscription,picture,note,Geohack,lat,lon
0,Hamburg,Altona,Altona-Altstadt,Amundsenstraße 14A,Max Cohn,Hier wohnte Max Cohn Jg. 1872 deportiert 1942 ...,NaN,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-Al...,53.546928,9.948774
1,Hamburg,Altona,Altona-Altstadt,Amundsenstraße 24,Friedrich W. Hoffmann,Hier wohnte Friedrich W. Hoffmann Jg. 1889 im ...,NaN,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-Al...,53.546698,9.947123
2,Hamburg,Altona,Altona-Altstadt,Behnstraße 17,Alfred Rosentreter,Hier wohnte Alfred Rosentreter Jg. 1934 deport...,NaN,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-Al...,53.547465,9.94201
3,Hamburg,Altona,Altona-Altstadt,Behnstraße 17,Betty Rosentreter,Hier wohnte Betty Rosentreter Jg. 1926 deporti...,NaN,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-Al...,53.547465,9.94201
4,Hamburg,Altona,Altona-Altstadt,Behnstraße 17,Fanny Rosentreter geb. Weinberger,Hier wohnte Fanny Rosentreter geb. Weinberger ...,NaN,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-Al...,53.547465,9.94201
...,...,...,...,...,...,...,...,...,...,...,...
257,Hamburg,Wandsbek,15,Hasselwisch 5,Georg Mewes,Hier wohnte Georg Mewes Jg. 1909 verhaftet 194...,NaN,Eintrag auf stolpersteine-hamburg.de,Fix Geohack,Fix lat,Fix lon
258,Hamburg,Wandsbek,15,Lottbeker Weg 13,Dorothea Gatzenmeyer geb. Jürs,Hier wohnte Dorothea Gatzenmeyer geb. Jürs Jg....,NaN,Eintrag auf stolpersteine-hamburg.de,Fix Geohack,Fix lat,Fix lon
259,Hamburg,Wandsbek,15,Lottbeker Weg 13,Louis Gatzenmeyer,Hier wohnte Louis Gatzenmeyer Jg. 1905 im Wide...,NaN,Eintrag auf stolpersteine-hamburg.de,Fix Geohack,Fix lat,Fix lon
260,Hamburg,Wandsbek,15,Lottbeker Weg 24,Chaja Rywa Balck geb. Skop,Hier wohnte Chaja Rywa Balck geb. Skop Jg. 189...,NaN,Eintrag auf stolpersteine-hamburg.de Stolperst...,Fix Geohack,Fix lat,Fix lon


In [483]:
df_hamburg.to_csv('stolpersteine_hamburg.csv', index = False)

# Analysis

In [2]:
import pandas as pd

df = pd.read_csv('stolpersteine_hamburg.csv')
df.head()

,city,borough,district,address,name,inscription,picture,note,Geohack,lat,lon,deported,murdered,Auschwitz
0,Hamburg,Altona,Altona-Altstadt,Amundsenstraße 14A,Max Cohn,Hier wohnte Max Cohn Jg. 1872 deportiert 1942 ...,NaN,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-Al...,53.546928,9.948774,True,True,False
1,Hamburg,Altona,Altona-Altstadt,Amundsenstraße 24,Friedrich W. Hoffmann,Hier wohnte Friedrich W. Hoffmann Jg. 1889 im ...,NaN,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-Al...,53.546698,9.947123,False,True,False
2,Hamburg,Altona,Altona-Altstadt,Behnstraße 17,Alfred Rosentreter,Hier wohnte Alfred Rosentreter Jg. 1934 deport...,NaN,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-Al...,53.5474654,9.9420098,True,True,False
3,Hamburg,Altona,Altona-Altstadt,Behnstraße 17,Betty Rosentreter,Hier wohnte Betty Rosentreter Jg. 1926 deporti...,NaN,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-Al...,53.5474654,9.9420098,True,True,False
4,Hamburg,Altona,Altona-Altstadt,Behnstraße 17,Fanny Rosentreter geb. Weinberger,Hier wohnte Fanny Rosentreter geb. Weinberger ...,NaN,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-Al...,53.5474654,9.9420098,True,True,False


In [16]:
df_small = df.drop(columns= ['picture','Geohack','note','deported','murdered','Auschwitz'])

In [17]:
df_small.to_csv('stolpersteine_hamburg_small.csv', index = False)

#### Get number per borough

In [19]:
df['borough'].value_counts()

borough
Eimsbüttel       2922
Hamburg-Nord     1512
Hamburg-Mitte    1459
Altona            526
Wandsbek          262
Harburg           257
Bergedorf          34
Name: count, dtype: int64

#### Get number per district

In [21]:
## Fix district names
df['district'].value_counts()

district
Rotherbaum_(Stra%C3%9Fen_A%E2%80%93H)      749
Harvestehude_(Stra%C3%9Fen_I%E2%80%93Z)    596
Rotherbaum_(Stra%C3%9Fen_I%E2%80%93Z)      511
Eimsb%C3%BCttel                            477
Neustadt                                   477
                                          ... 
Sinstorf                                     1
Hausbruch                                    1
Altenwerder                                  1
Finkenwerder                                 1
Marmstorf                                    1
Name: count, Length: 79, dtype: int64

#### Get number per name - first split name into name and surname ...

In [ ]:
## be aware that there are also maiden names ...
## df['name'].value_counts()

#### Get year of birth from inscription ("Inschrift")

In [496]:
#df['year_of_birth'] = df['inscription'].str.extract("Jg. (\d*)", expand=False).astype(float)

In [497]:
#df['year_of_birth'] = df['year_of_birth'].astype(int)

#### Get year of deportation from inscription

In [498]:
#df['deported'] = df['inscription'].str.extract("deportiert (\d*)", expand=False).astype(float)

#### Get age when deported

In [499]:
#df['age_when_deported']= df(df['deportiert'] - df['Jahrgang'])

#### Get deported

In [505]:
df['deported'] = df['inscription'].str.contains("deportiert")

In [512]:
df['deported'].value_counts()

deported
True     4672
False    2298
Name: count, dtype: int64

#### Get Polenaktion

In [500]:
#df['Polenaktion'] = df['inscription'].str.extract("'Polenaktion' (\d*)", expand=False)

#### Get murderd ("ermordet")

In [506]:
df['murdered'] = df['inscription'].str.contains("ermordet")

In [511]:
df['murdered'].value_counts()

murdered
True     4879
False    2091
Name: count, dtype: int64

#### Get Auschwitz

In [507]:
df['Auschwitz'] = df['inscription'].str.contains("Auschwitz")

In [510]:
df['Auschwitz'].value_counts()

Auschwitz
False    5763
True     1207
Name: count, dtype: int64

In [508]:
df.head()

,city,borough,district,address,name,inscription,picture,note,Geohack,lat,lon,deported,murdered,Auschwitz
0,Hamburg,Altona,Altona-Altstadt,Amundsenstraße 14A,Max Cohn,Hier wohnte Max Cohn Jg. 1872 deportiert 1942 ...,NaN,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-Al...,53.546928,9.948774,True,True,False
1,Hamburg,Altona,Altona-Altstadt,Amundsenstraße 24,Friedrich W. Hoffmann,Hier wohnte Friedrich W. Hoffmann Jg. 1889 im ...,NaN,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-Al...,53.546698,9.947123,False,True,False
2,Hamburg,Altona,Altona-Altstadt,Behnstraße 17,Alfred Rosentreter,Hier wohnte Alfred Rosentreter Jg. 1934 deport...,NaN,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-Al...,53.5474654,9.9420098,True,True,False
3,Hamburg,Altona,Altona-Altstadt,Behnstraße 17,Betty Rosentreter,Hier wohnte Betty Rosentreter Jg. 1926 deporti...,NaN,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-Al...,53.5474654,9.9420098,True,True,False
4,Hamburg,Altona,Altona-Altstadt,Behnstraße 17,Fanny Rosentreter geb. Weinberger,Hier wohnte Fanny Rosentreter geb. Weinberger ...,NaN,Eintrag auf stolpersteine-hamburg.de,pagename=Liste_der_Stolpersteine_in_Hamburg-Al...,53.5474654,9.9420098,True,True,False


In [509]:
df.describe()

,city,borough,district,address,name,inscription,picture,note,Geohack,lat,lon,deported,murdered,Auschwitz
count,6972,6972,6971,6972,6971,6970,1524,6966,6972,6971,6971,6970,6970,6970
unique,1,7,79,2982,6610,6953,38,433,6707,4291,4192,2,2,2
top,Hamburg,Eimsbüttel,Rotherbaum_(Stra%C3%9Fen_A%E2%80%93H),Essener Straße 54,–,Hier wohnte Siegbert Leser Jg. 1888 deportiert...,Fix picturelist,Eintrag auf stolpersteine-hamburg.de,Fix Geohack,Fix lat,Fix lon,True,True,False
freq,6972,2922,749,50,17,2,1459,4740,262,262,262,4672,4879,5763


In [513]:
df.to_csv('stolpersteine_hamburg.csv', index = False)

In [ ]:
#page_title = []

#for st in stadtteile:
#        html = requests.get(url).text
#        soup = BeautifulSoup(html)
#        title = soup.select("firstHeading")
#    page_title.append(title)
        print(title.text[:1])
    
#df['Stadtteil'] = stadtteil